<a href="https://colab.research.google.com/github/JanEggers-hr/chatgpt-playground/blob/main/jans_gpt_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# chatGPT for the rest of us! (English Version)

*v1.4* - A Colab notebook for chatting with the GPT large language models, offering the additional settings from the playground All you need is an API token.

## How to use this

If you are still on Github right now, click the "Open in Colab" Button top left - this copies the code to a Google Colab virtual environment. Which is a computer in the cloud **where Google allows you to run simple Python programs free of charge**. (You will need to login with some kind of Google ID, be it GMail, Android, or Youtube.

All there is to do: Click the small "play" button symbol below these paragraphs to run the code, wait for it to load the first batch of programs, then copy an API token into the slot.

To avoid losing the connection to the Colab environment, I recommend using [Colab Automatic Clicker (Firefox)](https://addons.mozilla.org/en-US/firefox/addon/colab-automatic-clicker/) or[Colab Auto Reconnect (Chrome)](https://chrome.google.com/webstore/detail/colab-auto-reconnect/ifilpgffgdbhafnaebocnofaehicbkem)

## The Chatbot

You may adjust these parameters:
- **Temperature** controls the amount of chance the model allows when selecting the next tokens to answer.
- **Model** determins which language model you use. GPT3.5 is the cheaper standard model, GPT-4 ist the most powerful model at the time being. GPT-4 32k is for very long input contexts (up to approx. 25.000 words).
- **Stop Token** is used to structure sample input. Start with a prompt, then a stop token, then a query-answer pair, then another stop token, then another query-answer pair.
- **Max-Token** makes the model stop after generating these tokens. Set it very low if you want to get short answert like Yes or No.
- **System** is the bot's personality; the way it is supposed to behave. Bots may escape the instructions in their system prompt, especially GPT3.5.


If you wish to use the chatbot, **you will need to copy a valid OpenAI API token code into the window when prompted.**


## Click the play symbol below to run the code!

In [ ]:
#@title
import requests
import json
import math
import markdown

# ipywidgets ist schon installiert
import ipywidgets as widgets
from IPython.display import display

# Modelle und Kosten definieren
# Kosten in US-Dollar je 1000 Tokens
models_token_info = {'gpt-3.5-turbo': {
                                        'pricing': 0.002,
                                        'max_tokens': 4096
                                      },
          'gpt-4': {
                                        'pricing': 0.03,
                                        'max_tokens': 8192
                                      },
          'gpt-4-32k': {
                                        'pricing': 0.06,
                                        'max_tokens': 32768
                                      }}

textbox_max_tokens = widgets.Text(
    value='0',
    placeholder='0',
    description='Max. Token:',
)

area_system = widgets.Textarea(
    value = 'You are ChatGPT, a helpful AI assistant. \
You try to solve every tasp presented to you, proceeding step by step.\n',
    rows=10,
    description = 'System:'
)

# Temperatur-Slider
slider_temperature = widgets.FloatSlider(
    value=0.7,
    min=0,
    max=1,
    step=0.1,
    description='Temperature:',
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

# Best-of-Slider
slider_bestof = widgets.IntSlider(
    value=1,
    min=1,
    max=4,
    description='Best Of:',
    orientation='horizontal',
    readout=True
)


dropdown_model = widgets.Dropdown(
    # Nimm die oben definierte Preisliste als Basis
    options=list(models_token_info.keys()),
    value=list(models_token_info.keys())[0],
    description='Model:',
)

textbox_stop = widgets.Text(
    value='###\n',
    placeholder="###",
    description="Stop Token:"
)

# Funktion wird bei Veränderung ausgeführt
def update_params(change):
    global temperature
    global max_tokens
    global system_prompt
    global model
    global stoptokens
    global best_of
    temperature = slider_temperature.value
    best_of = slider_bestof.value
    # Token-Obergrenze umrechnen
    try:
        max_tokens = int(textbox_max_tokens.value)
        if max_tokens == 0:
            max_tokens = None
    except ValueError:
        max_tokens = None
    textbox_max_tokens.value = f'{max_tokens}'
    system_prompt = area_system.value
    model = dropdown_model.value
    stoptokens = textbox_stop.value
    if (stoptokens == ""):
      stoptokens = None

# Verbinde die Widgets mit der Funktion zur Verarbeitung der Werte
textbox_max_tokens.observe(update_params, 'value')
slider_temperature.observe(update_params, 'value')
slider_bestof.observe(update_params, 'value')
area_system.observe(update_params, 'value')
textbox_stop.observe(update_params, 'value')
dropdown_model.observe(update_params, 'value')

# Bisschen breiter anzeigen
textbox_max_tokens.layout.width = '200px'
dropdown_model.layout.width = '300px'
area_system.layout.width = '600px'

# Vorbereitungen für die Einstellungen sind getan - jetzt die OpenAI-Libraries
update_params(0)
print("Widgets generated")

# Tokenizer Tiktoken einbinden
!pip install -q tiktoken
import tiktoken
print("Tokenizer tiktoken loaded")

# OpenAI-API-Library einbinden
!pip install -q openai
import openai
print("OpenAI-API-Library loaded")

def on_chatbot_reset_clicked(button):
    global previous_messages
    global spent_tokens
    global spent_dollars
    previous_messages = []
    spent_tokens = 0
    spent_dollars = 0.00
    chatbot_output_area.value = ''

def chatbot(prompts):
    # Prompt
    response = openai.ChatCompletion.create(
        model=model,
        messages=prompts,
        n=1,
#        best_of = best_of,
        stop=stoptokens,
        max_tokens=max_tokens,
        temperature=temperature,
        stream = True
    )
    return response

text_tokens = widgets.HTML(
    value = '<b>Tokens used</b>: 0 ($0.00)'
)

# Define the widget for displaying token usage
def update_token_usage_widget(value):
    global spent_tokens
    global spent_dollars
    spent_tokens += value
    spent_dollars += pricing(value)

    token_usage_text = f'<b>Tokens used</b> {spent_tokens} ($ {spent_dollars:.3f}) '
    text_tokens.value = token_usage_text

chatbot_output_area = widgets.HTML(
    value='',
    description='Dialog:',
    layout=widgets.Layout(width='100%')
)
# Hilfsfunktion: Token berechnen
def calculate_tokens(string: str) -> int:
    """Returns the number of tokens in a text string."""
    # cl100k_base ist der Tokenizer für Davinci, GPT-3 und GPT-4
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Ausgaben von GPT formatieren:
# - \n in <br> umsetzen
# - Codeblöcke mit <pre><code> beginnen und abschließen

import re
import markdown

def gptparse(text):
    # Preprocessing: <br> durch \n ersetzen,
    # dann umwandeln
    #
    # (Braucht eine Extension, um MD in HTML zu verstehen)
    htmltext = markdown.markdown(text.replace("\n",""),extensions=['md_in_html'])
# alte Codeblock-Umwandlung, in case it does not work
#    pattern =  r'\`\`\`(?P<text>[^*]+)\`\`\`'
#    htmltext = re.sub(pattern, r'<code><pre>\g<text></pre></code>', text)
    return htmltext

def gptparse2(previous_messages):
    text = ""
    for item in previous_messages:
        if item["role"] == "assistant":
            p_text = '<p style="font-family: Verdana; font-style: italic;" markdown="1">'
            p_text += '<b>Chatbot: </b>'
            p_text += item["content"]
            p_text += '</p>'
            text += markdown.markdown(p_text,extensions=['md_in_html','extra','codehilite','nl2br'])
        if item["role"] == "user":
            p_text = '<p style="font-family: Verdana;" markdown="1">'
            p_text += '<b>Du: </b>'
            p_text += item["content"]
            p_text += '</p>'
            text += markdown.markdown(p_text,extensions=['md_in_html','extra','codehilite','nl2br'])
    return text

# Define the function to be called when the chatbot is used
def on_chatbot_button_clicked(button):
    global chatbot_output
    # Get the user's input and display it
    user_input = user_text.value
    user_text.value = ''
    # Generate a response from the chatbot
    chatbot_output_area.value += f'<p style="font-family: Verdana;" markdown="1"><b>Du</b>: {user_input}</p>'
    messages = [
            {"role": "system", "content": system_prompt},
            *previous_messages,
            {"role": "user", "content": user_input},
        ]
    chatbot_output_area.value += '<p style="font-family: Verdana; font-style: italic;" markdown="1"><b>Chatbot: </b>'
    # Stream-Objekt mit der Antwort
    chatbot_response = chatbot(messages)
    collected_messages = []   # braucht man nicht zwingend
    # Anzahl von Tokens mit der User-Frage initiieren
    chunk_tokens = calculate_tokens(user_input)
    # Iteriere über die Chunks (die Brocken )
    for chunk in chatbot_response:
        chunk_message = chunk['choices'][0]['delta']  # extract the message
        collected_messages.append(chunk_message)  # save the event response
        # Ausgabefenster: Neuen Chunk anhängen
        chatbot_output = chunk_message.get('content', '')
        # /n durch <br> ersetzen
        chatbot_output_area.value += re.sub('\r?\n','<br>',chatbot_output)
        update_token_usage_widget(calculate_tokens(chatbot_output))
    # Stream-HTML-Block abschließen...
    chatbot_output_area.value += '</p>'
    # Antwort komplett in die Chathistorie aufnehmen
    chatbot_output = ''.join([m.get('content', '') for m in collected_messages])
    previous_messages.extend([
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": chatbot_output},
    ])
    # ... und Code neu formatieren
    chatbot_output_area.value = gptparse2(previous_messages)
    # ...und die Länge in Tokens berechnen und ergänzen

def pricing(tokens):
    price = models_token_info.get(model)['pricing']
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)


# Define the chatbot input and output widgets
user_text = widgets.Text(
    placeholder='...',
    description='You:',
    layout=widgets.Layout(width='60%'),
)

# Definiere den Absenden-Button und binde ihn an on_chatbot_button_clicked
chatbot_button = widgets.Button(
    description='Send',
    layout=widgets.Layout(width='15%'),
)
chatbot_reset = widgets.Button(
    description = 'Reset',
    layout=widgets.Layout(width='15%')
)
##### Der eigentliche Code! #####

from getpass import getpass
key_needed = True
while key_needed:
    openai.api_key = getpass("Enter OpenAI API Key here: ")
    try:
        # Testweise Modelle abfragen
        models = openai.Model.list()['data']
        # Erfolg?
        print("API Key is valid!")
        key_needed = False
    except:
        print("Query error - invalid API key maybe?")
previous_messages = []
spent_tokens = 0        # Wie viele Tokens wurden bisher über die API abgefragt?
spent_dollars = 0.00    # Zu welchem Preis?
codeblock = False       # Hat Ausgabe eines Codeblocks begonnen?

# Die Einstellungs-Widgets anzeigen
# Setzt die globalen Variablen temperature, system_prompt, api_key, model, stoptokens
display(slider_temperature,
#        slider_bestof,
        dropdown_model,
        textbox_stop,
        textbox_max_tokens,
        area_system)

# Die Eingabefelder registrieren

chatbot_button.on_click(on_chatbot_button_clicked)
chatbot_reset.on_click(on_chatbot_reset_clicked)
# Abschicken auch durch Return in der user_text Box
user_text.on_submit(on_chatbot_button_clicked)


# Display the chatbot widgets
display(text_tokens)
display(chatbot_output_area, user_text, chatbot_button, chatbot_reset)



Widgets eingerichtet.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.2 MB/s eta 0:00:00
Tokenizer tiktoken geladen.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.0 MB/s eta 0:00:00
OpenAI-API-Library geladen.
OpenAI-API-Key eingeben: ··········
API-Key gültig!


FloatSlider(value=0.7, description='Temperatur:', max=1.0)

Dropdown(description='Modell:', layout=Layout(width='300px'), options=('gpt-3.5-turbo', 'gpt-4', 'gpt-4-32k'),…

Text(value='###\n', description='Stop-Token:', placeholder='###')

Text(value='None', description='Max. Token:', layout=Layout(width='200px'), placeholder='0')

...

HTML(value='<b>Verbrauchte Token</b>: 0 ($0.00)')

HTML(value='', description='Dialog:', layout=Layout(width='100%'))

Text(value='', description='Du:', layout=Layout(width='60%'), placeholder='...')

Button(description='Absenden', layout=Layout(width='15%'), style=ButtonStyle())

Button(description='Reset', layout=Layout(width='15%'), style=ButtonStyle())

### Known issues

- Might drop the last couple of characters if you type too fast, and press Enter too fast.
- The token usage calculation is an approximation as streamed return data gives no usage information